In [395]:
#!pip install nltk

In [396]:
#!pip install beautifulsoup4

In [397]:
#!pip install lxml

In [398]:
#!pip install xlrd

In [399]:
#nltk.download()

In [400]:
from bs4 import BeautifulSoup

In [401]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

import re
import scipy as s

otto=pd.read_excel('leuphana_datensatz.xlsx',encoding='utf8')
otto.head(7)

,searchterm,product_id,product_name,product_description,product_sellingpoints
0,waschmaschine,360700745,"Privileg Waschmaschine Toplader PWT 3525, A+, ...",Die PWT 3525 ist eine Toplader-Waschmaschine v...,"<sellingPoints xmlns:rp=""http://www.otto.de/sc..."
1,vans,555220815,Vans T-Shirt,Das T-Shirt von Vans begeistert mit seinem aut...,"<sellingPoints xmlns:rp=""http://www.otto.de/sc..."
2,schreibtisch,403881979,Schreibtisch »8048«,Schreibtisch mit ergonomisch geformter Tischpl...,"<sellingPoints xmlns:rp=""http://www.otto.de/sc..."
3,trampolin,495982357,"4Uniq Trampolin, Ø 305 cm, inkl. Sicherheitsne...","4Uniq Trampolin, Ø 305 cm, inkl. Sicherheitsne...","<sellingPoints xmlns:rp=""http://www.otto.de/sc..."
4,spiegelschrank,536509117,"Spiegelschrank »Junior« Breite 31,5 cm",<p><b>Produktdetails</b></p><ul><li>Serienname...,"<sellingPoints xmlns:rp=""http://www.otto.de/sc..."
5,sekretär,495873227,Loberon Tischsekretär »Lovell«,Tischsekretär Lovell<br> <br> Der hölzerne Tis...,"<sellingPoints xmlns:rp=""http://www.otto.de/sc..."
6,schreibtisch,377847434,Wimex Schreibtisch,Modernes Jugendzimmerprogramm. Pflegeleichte K...,"<sellingPoints xmlns:rp=""http://www.otto.de/sc..."


In [402]:
def clean (x):    
    #change to lower case
    x=x.str.lower()

    #Remove nulls
    x = x[~pd.isnull(x)]
    x.index=range(len(x))


    z = pd.Series(s.zeros(len(x)))
    for i in range(len(x)):
        z[i] = re.sub(r'<.*?>',' ',x[i])
        z[i] = re.sub(r'[?|$|*|:|>|<|=|,|\|/|"|(|)|.|+|»|«|!]',' ',z[i])
    

    #tokenize
    y = pd.Series(s.zeros(len(z)))
    for i in range(len(z)):
        y[i] = word_tokenize(z[i])
       
    #remove words less than 2 characters
    for j in range(len(y)):
        for i in range(len(y[j])):
            if len(y[j][i])<3:
                y[j][i]=""
        
    #remove stop and exclude words
    exclude_words= set(('ja','nein','cm','mehr','oben','','bitte','sellingpoints'))
    stop = set(stopwords.words("german"))
    stops=stop|exclude_words

    w= pd.Series(s.zeros(len(y)))
    for j in range(len(y)):
        w[j]=[i for i in y[j] if i not in stops]
    return w

In [403]:
#########Keywords by search term
setterm='ski'
#change to lower case
otto.searchterm=otto.searchterm.str.lower()

In [404]:
#clean description and product name data
clean_description=clean(otto[otto.searchterm==setterm]['product_description'])
clean_product_name=clean(otto[otto.searchterm==setterm]['product_name'])

In [405]:
#clean selling points data
clean_selling_point=clean(otto[otto.searchterm==setterm]['product_sellingpoints'])

In [406]:
#join outputs
join_descriptions = clean_description.apply(pd.Series).stack().reset_index(drop=True)

join_product_name = clean_product_name.apply(pd.Series).stack().reset_index(drop=True)

join_selling_point = clean_selling_point.apply(pd.Series).stack().reset_index(drop=True)

join_total_output = pd.concat([join_descriptions,join_product_name,join_selling_point])

In [407]:
#Show results(description) by column
results_description = pd.crosstab(join_descriptions, columns="count")  

results_description.sort_values(by=['count'], ascending=False).head(7)
#results_description 

col_0,count
row_0,
100,61
polyester,49
ski,47
lego®,45
ski-,45
details,44
uvex,44


In [408]:
#Show results(product name) by column


results_product = pd.crosstab(join_product_name, columns="count")  
results_product.sort_values(by=['count'], ascending=False).head(7)

col_0,count
row_0,
ski,72
snowboardhelm,28
uvex,21
trigema,12
alpina,9
langarm,8
sport,8


In [409]:
#Show results(selling point)  by column
results_selling_points = pd.crosstab(join_selling_point, columns="count")  
results_selling_points.sort_values(by=['count'], ascending=False).head(7)

col_0,count
row_0,
2016,31
modelljahr,30
100,18
belüftungssystem,13
innenausstattung,12
abnehmbare,12
kopfbandhalterung,11


In [410]:
# Results total output
results = pd.crosstab(join_total_output, columns="count")  
results.sort_values(by=['count'], ascending=False).head(12)

col_0,count
row_0,
ski,123
100,79
uvex,65
lego®,59
polyester,58
material,48
ski-,47
system,47
details,44


In [411]:
#####################################NEW TEILEEEEEEEEEEEEEE

In [412]:
##############DO SIMPLE
def doall (setterm,method,N):
    otto.searchterm=otto.searchterm.str.lower()
    #clean description and product name data
    clean_description=clean(otto[otto.searchterm==setterm]['product_description'])
    clean_product_name=clean(otto[otto.searchterm==setterm]['product_name'])
    #clean selling points data
    clean_selling_point=clean(otto[otto.searchterm==setterm]['product_sellingpoints'])
    #join outputs
    join_descriptions = clean_description.apply(pd.Series).stack().reset_index(drop=True)
    join_product_name = clean_product_name.apply(pd.Series).stack().reset_index(drop=True)
    join_selling_point = clean_selling_point.apply(pd.Series).stack().reset_index(drop=True)
    join_total_output = pd.concat([join_descriptions,join_product_name,join_selling_point])
    # results by column
    results_description = pd.crosstab(join_descriptions, columns="count")  
    # results by column
    results_product = pd.crosstab(join_product_name, columns="count")  
    # results by column
    results_selling_points = pd.crosstab(join_selling_point, columns="count")  
    # Results total output
    results = pd.crosstab(join_total_output, columns="count")
    
    if method=='all': 
        return results.sort_values(by=['count'], ascending=False).head(N)
    elif method=='name':
        return results_product.sort_values(by=['count'], ascending=False).head(N)
    elif method=='description':
        return results_description.sort_values(by=['count'], ascending=False).head(N)
    elif method=='sellingpoint':
        return results_selling_points.sort_values(by=['count'], ascending=False).head(N)

In [413]:
doall('acer','name',10)

col_0,count
row_0,
acer,234
zoll,97
notebook,70
intel®,65
speicher,62
core™,53
beamer,48
aspire,46
display,35


In [414]:
termlist=pd.Series(pd.crosstab(otto.searchterm,columns='count')['count'].index)

In [415]:
allresult=list()
for i in range(len(termlist)):
    allresult.append(doall(termlist[i],'name',15))

In [416]:
allresult

[col_0     count
 row_0          
 3ds         154
 nintendo    117
 new          28
 spiel        27
 mario        16
 pyramide     15
 software     15
 pokémon      12
 lego         11
 selects      11
 hama          9
 super         9
 warner        8
 games         8
 the           8, col_0             count
 row_0                  
 neff                 53
 wandhaube            44
 siemens              42
 kopffreihaube        37
 aktivkohlefilter     37
 kohlefilter          35
 bosch                34
 amica                26
 hanseatic            20
 breite               20
 serie                20
 leistung             20
 inselhaube           16
 starterset           15
 flachschirmhaube     15, col_0        count
 row_0             
 jayz            32
 toy             23
 joy             23
 ledergürtel     22
 armband         19
 firetti         18
 set             17
 chronograph     17
 stück           17
 quarzuhr        16
 beanie          14
 gürtel          13
 heli 

In [417]:
resultdf=pd.DataFrame()
for i in range(len(allresult)):
    resultdf=pd.concat([resultdf,pd.DataFrame(allresult[i])],ignore_index=True,axis=1)

resultdf.columns=termlist

In [418]:
resultdf.to_csv('./result.csv',encoding='utf-8')